In [2]:
import requests
from bs4 import BeautifulSoup, Tag
import re
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import os
from glob import glob
import ast
import csv
import random
from datetime import datetime, timedelta
from math import ceil
import time

### Permit Data Register Parsing

In [32]:
# Константи
BASE_URL = "https://e-construction.gov.ua/document/optype=100/filter=780_2024-01-01_2024-12-31"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept-Language": "uk-UA,uk;q=0.9"
}

def extract_permit_doc_ids(html, page_num=0):
    """
    Extracts permit document IDs and registration numbers from the HTML content of a page.
    
    Args:
        html (str): The HTML content of the page.
        page_num (int): The page number being processed.
    
    Returns:
        list: A list of tuples containing doc_id, registration_number, and page_num.
    """
    soup = BeautifulSoup(html, "html.parser")
    results = []

    items = soup.select("div.dataset__item")
    if len(items) < 10:
        tqdm.write(f"⚠️ Warning: only {len(items)} items found on page {page_num}")

    for item in items:
        link = item.select_one("a.btn.btn-primary[href*='doc_id='][href*='optype=100']")
        doc_match = re.search(r"doc_id=(\d+)", link["href"]) if link else None
        doc_id = doc_match.group(1) if doc_match else None

        name_tag = item.select_one("h3.opendata__name")
        registration_number = name_tag.text.strip() if name_tag else ""

        if doc_id:
            results.append((doc_id, registration_number, page_num))

    return results

def fetch_permit_page(page, session, headers):
    """
    Fetches the HTML content of a specific permit page.
    
    Args:
        page (int): The page number to fetch.
        session (requests.Session): The session object for making HTTP requests.
        headers (dict): The HTTP headers to use for the request.
    
    Returns:
        tuple: A tuple containing the page number and the HTML content (or None if failed).
    """
    url = BASE_URL if page == 1 else f"{BASE_URL}/page={page}"
    headers["Referer"] = url
    try:
        response = session.get(url, headers=headers, timeout=5)
        response.raise_for_status()
        return page, response.text
    except requests.RequestException:
        return page, None

def scrape_permit_docs(start_page=1, end_page=1000, save_interval=1000, output_file="permit_documents.csv", max_workers=10):
    """
    Scrapes permit documents from multiple pages and saves the data to a CSV file.
    
    Args:
        start_page (int): The starting page number.
        end_page (int): The ending page number.
        save_interval (int): The number of records to save in each batch.
        output_file (str): The name of the output CSV file.
        max_workers (int): The maximum number of threads to use for concurrent requests.
    
    Returns:
        list: A list of pages that were skipped during scraping.
    """
    all_data = []
    skipped_pages = []
    column_names = ["doc_id", "registration_number_edessb", "page_num"]

    session = requests.Session()
    pages_parsed = 0
    pages = set(range(start_page, end_page + 1))
    try:
        existing_data = pd.read_csv(output_file)
        parsed_pages = set(existing_data["page_num"].unique())
        tqdm.write(f"⚡ Already parsed {len(parsed_pages)}")
        pages = pages-parsed_pages
    except (IndexError, FileNotFoundError, pd.errors.EmptyDataError):
        tqdm.write("🔹 Starting from scratch!")
        pd.DataFrame(columns=column_names).to_csv(output_file, index=False)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_page = {executor.submit(fetch_permit_page, page, session, HEADERS): page for page in pages}

        for future in tqdm(as_completed(future_to_page), total=len(future_to_page), desc="📄 Scraping pages"):
            page = future_to_page[future]
            try:
                page, html = future.result()
                if html is None:
                    skipped_pages.append(page)
                    continue

                data = extract_permit_doc_ids(html, page_num=page)
                all_data.extend(data)
                pages_parsed += 1
            except Exception as e:
                tqdm.write(f"⚠️ Error processing page {page}: {e}")
                skipped_pages.append(page)

            if len(all_data) >= save_interval * 12:
                df = pd.DataFrame(all_data, columns=column_names)
                df.to_csv(output_file, mode='a', index=False, header=False)
                tqdm.write(f"💾 Saved batch at page {page}")
                all_data = []

    if all_data:
        df = pd.DataFrame(all_data, columns=column_names)
        df.to_csv(output_file, mode='a', index=False, header=False)
        tqdm.write(f"✅ Final save of {len(all_data)} records.")

    tqdm.write(f"🔚 Completed! Skipped pages: {len(skipped_pages)}")

    # Retry skipped
    still_skipped_pages = []
    if skipped_pages:
        tqdm.write("🔄 Retrying failed pages...")
        for sk_page in tqdm(skipped_pages, desc="🔁 Retrying"):
            page, html = fetch_permit_page(sk_page, session, HEADERS)
            if html is None:
                tqdm.write(f"❌ Failed to retry page {sk_page}")
                still_skipped_pages.append(sk_page)
                continue
            data = extract_permit_doc_ids(html, page_num=sk_page)
            all_data.extend(data)
            df = pd.DataFrame(all_data, columns=column_names)
            df.to_csv(output_file, mode='a', index=False, header=False)
            all_data = []

    tqdm.write(f"🔚 Final skipped pages: {len(still_skipped_pages)}")
    return still_skipped_pages

In [33]:
# scrape_permit_docs(end_page=11472, output_file="permit_documents_3.csv", max_workers=10)

In [34]:
# parse register 3 times to get all data
df_1 = pd.read_csv('permit_documents_1.csv')
df_2 = pd.read_csv('permit_documents_2.csv')
df_3 = pd.read_csv('permit_documents_3.csv')

In [35]:
# Convert to sets for comparison
doc_ids_df1 = set(df_1['doc_id'].dropna())
doc_ids_df2 = set(df_2['doc_id'].dropna())
doc_ids_df3 = set(df_3['doc_id'].dropna())


# Total unique doc_id across all three
all_unique_doc_ids = len(doc_ids_df1 | doc_ids_df2 | doc_ids_df3)
print("Total unique doc_id across all:", all_unique_doc_ids)

# Shared in all three
shared_all = doc_ids_df1 & doc_ids_df2 & doc_ids_df3
print("Shared between all three:", len(shared_all))

# Unique to each
print("Unique to df_1:", len(doc_ids_df1 - doc_ids_df2 - doc_ids_df3))
print("Unique to df_2:", len(doc_ids_df2 - doc_ids_df1 - doc_ids_df3))
print("Unique to df_3:", len(doc_ids_df3 - doc_ids_df1 - doc_ids_df2))


Total unique doc_id across all: 132852
Shared between all three: 44162
Unique to df_1: 8985
Unique to df_2: 9530
Unique to df_3: 9762


In [36]:
almost_all_ids = pd.DataFrame({'doc_id': list(doc_ids_df1 | doc_ids_df2 | doc_ids_df3)})

# Збереження у CSV
# almost_all_ids.to_csv('permit_ids_final.csv', index=False)

### Permit Document Parsing

In [6]:
url_old = 'https://e-construction.gov.ua/document_detail/doc_id=2634101501580019683/optype=100'
url_new = 'https://e-construction.gov.ua/document_detail/doc_id=3295860232785233181/optype=100'
url = 'https://e-construction.gov.ua/document_detail/doc_id=3272199483311523749/optype=100'

In [7]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept-Language": "uk-UA,uk;q=0.9"
}

response = requests.get(url, headers=headers)
if response.status_code != 200:
    print(f"Error: {response.status_code}")
    exit()

# Parsing the HTML
soup = BeautifulSoup(response.text, "html.parser")

In [ ]:
def clean_text(text):
    """
    Cleans the input text by removing extra whitespace and replacing certain characters.

    Args:
        text (str): The input text to clean.

    Returns:
        str: The cleaned text.
    """
    return text.strip().replace(";", ",").replace("\n", " ").replace("\r", " ").replace("\t", " ")

def extract_named_tables(soup):
    """
    Extracts specific tables from the HTML soup based on predefined titles.

    Args:
        soup (BeautifulSoup): Parsed HTML content.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted table data.
    """
    table_titles = [
        "Відомості про замовника",
        "Замовники, які делегували свої повноваження",
        "Документ, на основі якого реєструється декларація",
        "Терміни будівництва"
    ]

    result_dict = {}

    for title in table_titles:
        h3 = soup.find("h3", string=lambda x: x and title in x)
        if not h3:
            continue

        next_div = h3.find_next_sibling()
        if next_div and "Інформацію не зазначено" in next_div.get_text():
            continue

        table = h3.find_next("table")
        if not table:
            continue

        thead = table.find("thead")
        tbody = table.find("tbody")
        if not thead or not tbody:
            continue

        headers = [th.get_text(strip=True) for th in thead.find_all("th")]
        first_row = tbody.find("tr")
        if not first_row:
            continue

        values = [td.get_text(strip=True) for td in first_row.find_all("td")]
        if not values:
            continue

        for h, v in zip(headers, values):
            result_dict[f"{title}_{clean_text(h)}"] = clean_text(v)

    return pd.DataFrame([result_dict])

def flatten_permit_data(parsed_dict):
    """
    Flattens a nested dictionary of permit data into a single-row DataFrame.

    Args:
        parsed_dict (dict): Nested dictionary of parsed permit data.

    Returns:
        pd.DataFrame: A DataFrame with flattened data.
    """
    flat_row = {}
    for section, entries in parsed_dict.items():
        if not entries:
            continue
        for entry in entries:
            data = entry.get('data', {})
            for key, value in data.items():
                col_name = f"{key}_{section}"
                flat_row[clean_text(col_name)] = clean_text(value)
    return pd.DataFrame([flat_row])

def parse_sections_with_subsections(soup):
    """
    Parses sections and subsections from the HTML soup.

    Args:
        soup (BeautifulSoup): Parsed HTML content.

    Returns:
        dict: A dictionary containing sections and their subsections with data.
    """
    result = {}

    for header in soup.select('h3.object-title'):
        section_name = clean_text(header.get_text())
        subsections = []
        # Початковий підрозділ — саме розділ (поки не зустріли <h5>)
        current_sub = None
        current_data = {}
        
        for elem in header.next_elements:
            # Якщо натрапили на новий розділ — зупиняємося
            if isinstance(elem, Tag) and elem.name == 'h3' and 'object-title' in elem.get('class', []):
                break
            # Якщо натрапили на підрозділ <h5> — зберігаємо попередній блок
            if isinstance(elem, Tag) and elem.name == 'h5':
                # зберегти, якщо є зібрані поля
                if current_data:
                    subsections.append({
                        'subsection': current_sub,
                        'data': current_data
                    })
                current_sub = clean_text(elem.get_text())
                current_data = {}
            # Якщо це інформаційний пункт — додаємо ключ/значення
            if isinstance(elem, Tag) and 'object-info-item' in elem.get('class', []):
                left = elem.select_one('.object-info_left')
                right = elem.select_one('.object-info_right')
                if left and right:
                    key = clean_text(left.get_text())
                    value = clean_text(right.get_text())
                    current_data[key] = value
        
        # Зберегти останній блок
        if current_data:
            subsections.append({
                'subsection': current_sub,
                'data': current_data
            })
        
        result[section_name] = subsections
    
    return result

def extract_fancytree_blocks(soup: BeautifulSoup) -> list[str]:
    """
    Extracts JavaScript blocks related to FancyTree from the HTML soup.

    Args:
        soup (BeautifulSoup): Parsed HTML content.

    Returns:
        list[str]: A list of extracted JavaScript blocks.
    """
    full_text = soup.decode()
    blocks = re.findall(
        r'head\.load\(\[.*?fancytree\.bundle\.js.*?\],function\(\)\{(.*?)\}\);',
        full_text,
        re.DOTALL
    )
    return blocks

def extract_parent_doc_id(js_text: str):
    """
    Extracts the parent document ID from a JavaScript block.

    Args:
        js_text (str): JavaScript block as a string.

    Returns:
        str or None: The extracted parent document ID, or None if not found.
    """
    match = re.search(r'"obj_comp_id"\s*:\s*"(\d+)"', js_text)
    if match:
        return match.group(1)
    return None

def final_function(soup):
    """
    Combines multiple parsing functions to extract and merge permit data.

    Args:
        soup (BeautifulSoup): Parsed HTML content.

    Returns:
        pd.DataFrame: A DataFrame containing the combined permit data.
    """
    info_table = extract_named_tables(soup)
    sections = parse_sections_with_subsections(soup)
    info_sections = flatten_permit_data(sections)
    blocks = extract_fancytree_blocks(soup)
    info_tep = {}
    for i, block in enumerate(blocks):
        info_tep[f"block_{i}"] = extract_parent_doc_id(block)

    # З'єднуємо таблиці info_table та info_sections по горизонталі
    base_df = pd.concat([info_table, info_sections], axis=1)

    # Витягуємо всі obj_comp_id у список
    tep_ids = list(filter(None, info_tep.values()))
    base_df["teps"] = [tep_ids]
    return base_df

In [9]:
# final_function(soup)

### TEP Parsing

In [11]:
url = "https://e-construction.gov.ua/document_detail_tep/doc_id=2895385515312285052"

In [12]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept-Language": "uk-UA,uk;q=0.9"
}

response = requests.get(url, headers=headers)
if response.status_code != 200:
    print(f"Error: {response.status_code}")
    exit()

# Parsing the HTML
soup = BeautifulSoup(response.text, "html.parser")

In [13]:
def parse_tep_info(soup):
    # Основна інформація
    basic_info = []
    for item in soup.select('.object-info-item'):
        label = item.select_one('.object-info_left span')
        value = item.select_one('.object-info_right span')
        if label and value:
            basic_info.append({
                'Розділ': 'Основна інформація про об’єкт',
                'Назва': label.text.strip(),
                'Значення': value.text.strip()
            })

    # ТЕП
    tep_info = []
    tep_table = soup.select_one('#tep + div table')
    if tep_table:
        for row in tep_table.select('tbody tr'):
            cols = row.find_all('td')
            if len(cols) >= 4:
                tep_info.append({
                    'Розділ': 'Техніко-економічні показники',
                    'Назва': cols[1].text.strip(),
                    'Значення': cols[3].text.strip(),
                    'Примітка': cols[4].text.strip() if len(cols) > 4 else ''
                })

    return pd.DataFrame(basic_info + tep_info)

### Get Permit Data

In [14]:
BASE_URL = "https://e-construction.gov.ua/document_detail/doc_id={}/optype=100"

def clean_text(text):
    return text.strip().replace(";", ",").replace("\n", " ").replace("\r", " ").replace("\t", " ")

def extract_named_tables(soup):
    table_titles = [
        "Відомості про замовника",
        "Замовники, які делегували свої повноваження",
        "Документ, на основі якого реєструється декларація",
        "Терміни будівництва"
    ]

    result_dict = {}

    for title in table_titles:
        h3 = soup.find("h3", string=lambda x: x and title in x)
        if not h3:
            continue

        next_div = h3.find_next_sibling()
        if next_div and "Інформацію не зазначено" in next_div.get_text():
            continue

        table = h3.find_next("table")
        if not table:
            continue

        thead = table.find("thead")
        tbody = table.find("tbody")
        if not thead or not tbody:
            continue

        headers = [th.get_text(strip=True) for th in thead.find_all("th")]
        first_row = tbody.find("tr")
        if not first_row:
            continue

        values = [td.get_text(strip=True) for td in first_row.find_all("td")]
        if not values:
            continue

        for h, v in zip(headers, values):
            result_dict[f"{title}_{clean_text(h)}"] = clean_text(v)

    return pd.DataFrame([result_dict])

def flatten_permit_data(parsed_dict):
    """
    Приймає словник з результатом parse_sections_with_subsections,
    повертає датафрейм: Ключ_Розділ
    """
    flat_row = {}
    for section, entries in parsed_dict.items():
        if not entries:
            continue
        for entry in entries:
            data = entry.get('data', {})
            for key, value in data.items():
                col_name = f"{key}_{section}"
                flat_row[clean_text(col_name)] = clean_text(value)
    return pd.DataFrame([flat_row])

def parse_sections_with_subsections(soup):
    """
    Парсить HTML сторінки ЄДЕССБ та витягує дані з кожного розділу <h3>,
    включно з підрозділами <h5>. Повертає структуру:

    {
      "Назва розділу": [
        {
          "subsection": "Назва підрозділу або None",
          "data": { key: value, ... }
        },
        ...
      ],
      ...
    }
    """
    result = {}

    for header in soup.select('h3.object-title'):
        section_name = clean_text(header.get_text())
        subsections = []
        # Початковий підрозділ — саме розділ (поки не зустріли <h5>)
        current_sub = None
        current_data = {}
        
        for elem in header.next_elements:
            # Якщо натрапили на новий розділ — зупиняємося
            if isinstance(elem, Tag) and elem.name == 'h3' and 'object-title' in elem.get('class', []):
                break
            # Якщо натрапили на підрозділ <h5> — зберігаємо попередній блок
            if isinstance(elem, Tag) and elem.name == 'h5':
                # зберегти, якщо є зібрані поля
                if current_data:
                    subsections.append({
                        'subsection': current_sub,
                        'data': current_data
                    })
                current_sub = clean_text(elem.get_text())
                current_data = {}
            # Якщо це інформаційний пункт — додаємо ключ/значення
            if isinstance(elem, Tag) and 'object-info-item' in elem.get('class', []):
                left = elem.select_one('.object-info_left')
                right = elem.select_one('.object-info_right')
                if left and right:
                    key = clean_text(left.get_text())
                    value = clean_text(right.get_text())
                    current_data[key] = value
        
        # Зберегти останній блок
        if current_data:
            subsections.append({
                'subsection': current_sub,
                'data': current_data
            })
        
        result[section_name] = subsections
    
    return result

def extract_fancytree_blocks(soup: BeautifulSoup) -> list[str]:
    """
    Витягує всі JS-блоки FancyTree з HTML. Блоки визначаються за формою:
    head.load([...fancytree.bundle.js...], function() { ... });

    Повертає список рядків з кожним знайденим блоком.
    """
    full_text = soup.decode()
    blocks = re.findall(
        r'head\.load\(\[.*?fancytree\.bundle\.js.*?\],function\(\)\{(.*?)\}\);',
        full_text,
        re.DOTALL
    )
    return blocks

def extract_parent_doc_id(js_text: str):
    match = re.search(r'"obj_comp_id"\s*:\s*"(\d+)"', js_text)
    if match:
        return match.group(1)
    return None

def final_function(soup):
    info_table = extract_named_tables(soup)
    sections = parse_sections_with_subsections(soup)
    info_sections = flatten_permit_data(sections)
    blocks = extract_fancytree_blocks(soup)
    info_tep = {}
    for i, block in enumerate(blocks):
        info_tep[f"block_{i}"] = extract_parent_doc_id(block)

    # З'єднуємо таблиці info_table та info_sections по горизонталі
    base_df = pd.concat([info_table, info_sections], axis=1)
    # Витягуємо всі obj_comp_id у список
    tep_ids = list(filter(None, info_tep.values()))
    base_df["teps"] = [tep_ids]
    return base_df

def fetch_page(doc_id, session, headers):
    url = BASE_URL.format(doc_id)
    headers["Referer"] = url
    try:
        response = session.get(url, headers=headers, timeout=5)
        response.raise_for_status()
        return doc_id, response.text
    except requests.RequestException as e:
        print(f"❌ Issue with doc_id {doc_id}: {e}")
        return doc_id, None

def parse_one(doc_id, session, headers):
    doc_id, html = fetch_page(doc_id, session, headers)
    if html is None:
        return None

    try:
        soup = BeautifulSoup(html, "html.parser")
        df = final_function(soup)

        # Ensure result is DataFrame
        if not isinstance(df, pd.DataFrame):
            print(f"⚠️ final_function did not return a DataFrame for {doc_id}")
            return None

        df["doc_id"] = doc_id
        return df.to_dict(orient="records")

    except Exception as e:
        print(f"⚠️ Error parsing doc_id {doc_id}: {e}")
        return None


def parse_permit_documents_parallel(ids_path: str, output_folder="parsed_chunks", max_workers=10, chunk_size=3000):
    # 1. Read doc_ids
    df_ids = pd.read_csv(ids_path, sep=';')
    doc_ids = df_ids['doc_id'].dropna().astype(str).tolist()

    # 2. Prepare session & headers
    session = requests.Session()
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "text/html,application/xhtml+xml",
    }

    os.makedirs(output_folder, exist_ok=True)
    all_records = []
    chunk_idx = 1

    # 3. Multithreaded processing
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(parse_one, doc_id, session, headers): doc_id
            for doc_id in doc_ids
        }

        for future in tqdm(as_completed(futures), total=len(futures), desc="Parsing documents"):
            result = future.result()
            if result:
                all_records.extend(result)

                # Check if we reached chunk size
                if len(all_records) >= chunk_size:
                    df_chunk = pd.DataFrame(all_records)
                    filename = os.path.join(output_folder, f"permit_chunk_{chunk_idx:04}.csv")
                    df_chunk.to_csv(filename, sep=';', index=False)
                    print(f"💾 Saved {len(df_chunk)} rows to {filename}")
                    all_records = []
                    chunk_idx += 1

    # Save remaining rows
    if all_records:
        df_chunk = pd.DataFrame(all_records)
        filename = os.path.join(output_folder, f"permit_chunk_{chunk_idx:04}.csv")
        df_chunk.to_csv(filename, sep=';', index=False)
        print(f"💾 Saved last {len(df_chunk)} rows to {filename}")

In [15]:
# parse_permit_documents_parallel("permit_ids_final.csv", output_folder="parsed_chunks")

In [16]:
money_number = 0
all_number = 0
for n in range(1, 46):
    try:
        file_path = f'parsed_chunks/permit_chunk_{n:04}.csv'  # formats n with 4 digits, e.g. 0001
        df = pd.read_csv(file_path, sep=';')
        money_number += df[~df['Загальна, тис. грн._Кошторисна вартість будівництва'].isna()].shape[0]
        all_number += df.shape[0]
    except FileNotFoundError:
        break


print(f"{money_number}/{all_number}")
print(money_number / all_number * 100)


39767/132843
29.935337202562422


In [17]:
def merge_parsed_chunks(input_folder="parsed_chunks", output_file="permit_page_merged.csv"):
    all_files = sorted(glob(os.path.join(input_folder, "*.csv")))
    all_dataframes = []

    for file in all_files:
        try:
            df = pd.read_csv(file, sep=';')
            all_dataframes.append(df)
        except Exception as e:
            print(f"⚠️ Failed to read {file}: {e}")

    if all_dataframes:
        merged_df = pd.concat(all_dataframes, axis=0, ignore_index=True, join='outer')
        merged_df.to_csv(output_file, sep=';', index=False)
        print(f"✅ Merged {len(all_dataframes)} chunks into {output_file} with {merged_df.shape[0]} rows and {merged_df.shape[1]} columns.")
    else:
        print("❌ No CSV files were loaded.")

In [18]:
merge_parsed_chunks()

✅ Merged 45 chunks into permit_page_merged.csv with 132843 rows and 91 columns.


### Selecting Data for TEP Download

In [19]:
raw_df = pd.read_csv("permit_page_merged.csv", sep=';')

C:\Users\dimit\AppData\Local\Temp\ipykernel_13148\1853856987.py:1: DtypeWarning: Columns (90) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv("permit_page_merged.csv", sep=';')


In [20]:
for col in raw_df.columns:
    if str(col).find('грн') != -1:
        print(col)

Загальна, тис. грн._Кошторисна вартість будівництва
За проектом, тис. грн._Кошторисна вартість будівництва
На буд. роботи, тис. грн._Кошторисна вартість будівництва
На машини, обладнання та інвентар, тис. грн._Кошторисна вартість будівництва


In [21]:
columns_to_check = [
    "Загальна, тис. грн._Кошторисна вартість будівництва",
    "За проектом, тис. грн._Кошторисна вартість будівництва",
    "На буд. роботи, тис. грн._Кошторисна вартість будівництва",
    "На машини, обладнання та інвентар, тис. грн._Кошторисна вартість будівництва"
]

at_least_one_notna = raw_df[columns_to_check].notna().any(axis=1).sum()
print(f"Кількість рядків з хоча б одним заповненим значенням: {at_least_one_notna}")

all_notna = raw_df[columns_to_check].notna().all(axis=1).sum()
print(f"Кількість рядків з усіма заповненими значеннями: {all_notna}")

count_general_filled = raw_df["Загальна, тис. грн._Кошторисна вартість будівництва"].notna().sum()
print(f"Кількість рядків, де 'Загальна, тис. грн._Кошторисна вартість будівництва' заповнена: {count_general_filled}")


Кількість рядків з хоча б одним заповненим значенням: 47609
Кількість рядків з усіма заповненими значеннями: 5220
Кількість рядків, де 'Загальна, тис. грн._Кошторисна вартість будівництва' заповнена: 39767


In [22]:
columns_to_check = [
    "Загальна, тис. грн._Кошторисна вартість будівництва",
    "За проектом, тис. грн._Кошторисна вартість будівництва",
    "На буд. роботи, тис. грн._Кошторисна вартість будівництва",
    "На машини, обладнання та інвентар, тис. грн._Кошторисна вартість будівництва"
]

df_filtered = raw_df[raw_df[columns_to_check].notna().any(axis=1)]

In [23]:
(df_filtered['teps'] != '[]').sum()

np.int64(42717)

In [24]:
df_filtered[df_filtered['teps'].str.contains(',')]['teps']

2            ['3475715044468066077', '3505494598975751463']
7            ['2895385515312285052', '3264099843236169450']
113          ['3409696146606524344', '3435001875726861471']
127          ['3399028885969437987', '3444595903778784609']
129          ['3219814599779943425', '3272336058095765008']
                                ...                        
132770    ['3418333938374935845', '102752710373728916819...
132780       ['3217550036523025927', '3287560134212977959']
132802    ['3474326641201645170', '548809842594968369583...
132826       ['3475749604224403369', '3475760174952612936']
132827       ['3511189562065945875', '3506954583450585008']
Name: teps, Length: 7015, dtype: object

In [25]:
(df_filtered['teps'].str.count(',')).value_counts()

teps
0    40594
1     7015
Name: count, dtype: int64

In [26]:
df_filtered[df_filtered['teps'].str.count(',') > 0][['teps', 'doc_id']]

,teps,doc_id
2,"['3475715044468066077', '3505494598975751463']",3509012527759492713
7,"['2895385515312285052', '3264099843236169450']",3272199483311523749
113,"['3409696146606524344', '3435001875726861471']",3468346468056695877
127,"['3399028885969437987', '3444595903778784609']",3455313825610335322
129,"['3219814599779943425', '3272336058095765008']",3277211245282853984
...,...,...
132770,"['3418333938374935845', '102752710373728916819...",3510615403648779855
132780,"['3217550036523025927', '3287560134212977959']",3290794653204350870
132802,"['3474326641201645170', '548809842594968369583...",3504733889669104912
132826,"['3475749604224403369', '3475760174952612936']",3475591300688905491


подивився на кілька докіментів, що мають кілька посилань на ТЕП, відповідно кілька розділів Об'єкти будівництва.

Треба брати Перший, він найповніший

In [28]:
def extract_first_tep(x):
    try:
        parsed = ast.literal_eval(x)
        if isinstance(parsed, list) and len(parsed) > 0:
            return parsed[0]
    except (ValueError, SyntaxError):
        pass
    return None

df_filtered.loc[:, 'first_tep'] = df_filtered['teps'].apply(extract_first_tep)

### Downloading TEPs

In [29]:
def long_to_wide(df, doc_id, include_prim=True):
    """
    Перетворює long DataFrame (Розділ, Назва, Значення, Примітка) у wide DataFrame з одним doc_id.

    Parameters:
        df (pd.DataFrame): Long формат з колонками 'Назва', 'Значення', 'Примітка'
        doc_id (str): Значення для doc_id, яке буде додане до всіх рядків
        include_prim (bool): Чи включати 'Примітка' як окремі колонки

    Returns:
        pd.DataFrame: Один рядок wide-формату з doc_id
    """
    df = df.copy()
    df["doc_id"] = doc_id

    wide_df = df.pivot_table(index="doc_id", columns="Назва", values="Значення", aggfunc="first")

    if include_prim and "Примітка" in df.columns:
        prim_df = df.pivot_table(index="doc_id", columns="Назва", values="Примітка", aggfunc="first")
        prim_df.columns = [f"{col} (примітка)" for col in prim_df.columns]
        wide_df = pd.concat([wide_df, prim_df], axis=1)

    wide_df = wide_df.reset_index()
    return wide_df

In [30]:
BASE_URL_TEP = "https://e-construction.gov.ua/document_detail_tep/doc_id={}"

def fetch_html(tep_doc_id, session, headers):
    url = BASE_URL_TEP.format(tep_doc_id)
    headers["Referer"] = url
    try:
        response = session.get(url, headers=headers, timeout=5)
        response.raise_for_status()
        return tep_doc_id, response.text
    except requests.RequestException as e:
        print(f"❌ Error for {tep_doc_id}: {e}")
        return tep_doc_id, None

def parse_one_tep(tep_doc_id, session, headers):
    tep_doc_id, html = fetch_html(tep_doc_id, session, headers)
    if html is None:
        return None

    try:
        soup = BeautifulSoup(html, "html.parser")
        long_df = parse_tep_info(soup)
        wide_df = long_to_wide(long_df, tep_doc_id)
        return wide_df
    except Exception as e:
        print(f"⚠️ Failed parsing tep_doc_id {tep_doc_id}: {e}")
        return None

def get_last_chunk_index(output_folder):
    existing_files = os.listdir(output_folder)
    chunk_nums = [int(re.search(r'(\d+)', f).group(1)) for f in existing_files if f.startswith("tep_chunk_") and f.endswith(".csv")]
    return max(chunk_nums, default=0)

def process_all_teps(df_with_teps, output_folder="tep_chunks", max_workers=10, chunk_size=500):
    tep_ids = df_with_teps["first_tep"].dropna().astype(str).unique().tolist()
    
    session = requests.Session()
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "text/html,application/xhtml+xml",
    }

    # 2. Отримаємо doc_id з tep_merged.csv (вже спарсені — треба виключити)
    try:
        df_existing = pd.read_csv("tep_merged.csv", sep=';')
        parsed_doc_ids = df_existing["doc_id"].dropna().astype(str).unique().tolist()
        print(f"📦 Loaded {len(parsed_doc_ids)} already parsed doc_ids from tep_merged.csv")
    except FileNotFoundError:
        parsed_doc_ids = []
        print("⚠️ File 'tep_merged.csv' not found. Proceeding with all TEPs.")

    # 3. Фільтруємо ті, що ще не були парсені
    tep_ids_to_parse = [tep_id for tep_id in tep_ids if tep_id not in parsed_doc_ids]
    print(f"🔍 Will parse {len(tep_ids_to_parse)} new TEPs")


    os.makedirs(output_folder, exist_ok=True)
    chunk_idx = get_last_chunk_index(output_folder) + 1
    current_chunk = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(parse_one_tep, tep_id, session, headers): tep_id
            for tep_id in tep_ids_to_parse
        }

        for future in tqdm(as_completed(futures), total=len(futures), desc="Parsing TEPs"):
            result = future.result()
            if result is not None:
                current_chunk.append(result)

                if len(current_chunk) >= chunk_size:
                    df_chunk = pd.concat(current_chunk, ignore_index=True)
                    filename = os.path.join(output_folder, f"tep_chunk_{chunk_idx:04}.csv")
                    df_chunk.to_csv(filename, sep=';', index=False)
                    print(f"💾 Saved {len(df_chunk)} rows to {filename}")
                    current_chunk = []
                    chunk_idx += 1

    # Save any remaining
    if current_chunk:
        df_chunk = pd.concat(current_chunk, ignore_index=True)
        filename = os.path.join(output_folder, f"tep_chunk_{chunk_idx:04}.csv")
        df_chunk.to_csv(filename, sep=';', index=False)
        print(f"💾 Saved last {len(df_chunk)} rows to {filename}")


In [31]:
process_all_teps(df_filtered)

C:\Users\dimit\AppData\Local\Temp\ipykernel_13148\2732572972.py:44: DtypeWarning: Columns (0,2,4,14,15,16,31,32,33,35,37,38,39,40,41,42,48,49,50,55,56,57,58,59,63,69,72,74,75,77,78,80,81,82,84,85,86,88,90,91,93,96,98,99,101,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,172,173,174,175,177,178,180,182,183,184,185,186,187,188,193,194,195,196,198,200,202,205,207,208,209,210,212,214,215,216,217,219,220,223,224,227,228,233,234,235,236,241,242,243,244,245,246,248,250,253,254,255,256,260,261,262,264,276,277,278,279,280,281,282,283,284,285,286,288,293,294,295,296,300,301,302,303,308,309,310,311,312,316,317,318,321,324,327,328,329,330,332,334,336,340,341,342,346,347,348,351,355,358,359,361,362,365,366,368,370,372,374,384,389,390,391,392,414,415,416,417,419,420,424,426,427,428,430,432,434,437,438,441,442,443,444,446,448,450,454,455,456,459,460,462,464,466,468,469,470,476,478,481,482,486,488,490,493,

📦 Loaded 35834 already parsed doc_ids from tep_merged.csv
🔍 Will parse 1 new TEPs
🔍 Will parse 1 new TEPs


Parsing TEPs: 100%|██████████| 1/1 [00:00<00:00,  4.65it/s]

💾 Saved last 1 rows to tep_chunks\tep_chunk_0074.csv


In [3]:
def merge_tep_chunks(input_folder="tep_chunks", output_file="tep_merged.csv"):
    all_files = sorted(glob(os.path.join(input_folder, "*.csv")))
    if not all_files:
        print("⚠️ No CSV files found.")
        return

    dataframes = []
    for file in all_files:
        try:
            df = pd.read_csv(file, sep=';')
            dataframes.append(df)
        except Exception as e:
            print(f"❌ Error reading {file}: {e}")

    if dataframes:
        merged_df = pd.concat(dataframes, ignore_index=True, sort=False)
        merged_df.to_csv(output_file, sep=';', index=False)
        print(f"✅ Merged {len(all_files)} chunks into {output_file} with {merged_df.shape[0]} rows.")
    else:
        print("❌ No data loaded from chunk files.")

merge_tep_chunks()

✅ Merged 74 chunks into tep_merged.csv with 35835 rows.
